BASIC DATA

In [2]:
# feeder_trace_latest_audit_with_rank.py  ✧  July 2025
"""
Workflow (HT-cable only)
========================
1. Load HTCABLE.csv, drop unused columns, remove fully-identical rows.
2. Trace every feeder edge-by-edge, annotate with RANK (distance from feeder start).
3. Add SOURCE_SS / DESTINATION_SS and split FROM_TO → FROM_SWITCH & TO_SWITCH.
4. Export to Excel – nothing from the energy-audit file is touched.
"""

from __future__ import annotations
import pandas as pd
from pathlib import Path
from typing import Dict, Tuple, List, Set, Optional

# ── CONFIG ────────────────────────────────────────────────────────────────────
INPUT_HT    = "/media/sagark24/New Volume/MERGE CDIS/2-Year-data/CLEANED_DATA/ht_cleaned.csv"                     # adjust path if needed
OUTPUT_PATH = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/11_KV_FINAL_HEALTH/AFINAL.csv"

FEEDER_ID_COL  = "FEEDERID"
SRC_SWITCH_COL = "SOURCE_SWITCH_ID"
DST_SWITCH_COL = "DESTINATION_SWITCH_ID"
SRC_LOC_COL    = "SOURCE_SSFL"
DST_LOC_COL    = "DESTINATION_SSFL"
FEEDER_ID_COL2  = "FEEDERID"
DATE = "DATECREATED"
# ─────────────────────────────────────────────────────────────────────────────
def _feeder_token(val: str | int | float | None) -> Optional[str]:
    if not isinstance(val, str):
        val = str(val) if val is not None else ""
    p = val.split("_")
    return p[2] if len(p) >= 3 and (p[1] == '11kV' or p[1]=='11Kv' or p[1]=='11KV') else None

def _feeder_token2(val: str | int | float | None) -> Optional[str]:
    if not isinstance(val, str):
        val = str(val) if val is not None else ""
    p = val.split("_")
    return val if len(p) >= 3 and (p[1] == '11kV' or p[1]=='11Kv' or p[1]=='11KV') else None


# 1️  LOAD & CLEAN HT-CABLE ---------------------------------------------------
ht = (pd.read_csv(Path(INPUT_HT).expanduser(), low_memory=False)
        
        .drop_duplicates())
ht["FEEDERID_FULL"] = ht[FEEDER_ID_COL2].apply(_feeder_token2).dropna()
ht["FEEDER_ID"] = ht[FEEDER_ID_COL].apply(_feeder_token)
for col in [SRC_SWITCH_COL, DST_SWITCH_COL, SRC_LOC_COL, DST_LOC_COL]:
    ht[col] = ht[col].astype(str)

# Ensure SOURCE_SS / DESTINATION_SS exist
if {"SOURCE_SS", "DESTINATION_SS"}.issubset(ht.columns):
    pass
else:
    ht["SOURCE_SS"] = ht[SRC_LOC_COL]
    ht["DESTINATION_SS"] = ht[DST_LOC_COL]

edge_cols = [SRC_SWITCH_COL, DST_SWITCH_COL, SRC_LOC_COL, DST_LOC_COL]
source_idx: Dict[Tuple[str, str], pd.DataFrame] = {
    (k[0], k[1]): g[edge_cols]
    for k, g in ht.groupby([SRC_LOC_COL, "FEEDER_ID"], sort=False)
}

# 2️  FEEDER TRACER -----------------------------------------------------------
def trace_feeder(fid: str) -> List[dict]:
    rows: List[dict] = []
    visited: Set[Tuple[str, str]] = set()

    start = ht[(ht[SRC_SWITCH_COL] == fid) & (ht["FEEDER_ID"] == fid)][edge_cols]
    queue = [(row, 0) for row in start.to_records(index=False).tolist()]

    while queue:
        (from_sw, to_sw, src_loc, dst_loc), rank = queue.pop(0)
        if (from_sw, to_sw) in visited:
            continue
        visited.add((from_sw, to_sw))

        rows.append({
            "FEEDER_ID": fid,
            "FROM_SWITCH": from_sw,
            "TO_SWITCH": to_sw,
         
            "SOURCE_LOCATION": src_loc,
            "DESTINATION_LOCATION": dst_loc,
            "RANK": rank
        })

        nxt = source_idx.get((dst_loc, fid))
        if nxt is not None and not nxt.empty:
            queue.extend([(row, rank + 1)
                          for row in nxt.to_records(index=False).tolist()])
    return rows

# 3️  TRACE ALL FEEDERS -------------------------------------------------------
trace_df = pd.DataFrame([row
                         for fid in ht["FEEDER_ID"].dropna().unique()
                         for row in trace_feeder(str(fid))])

# 4️  ADD SS COLUMNS ----------------------------------------------------------
trace_df = (trace_df.merge(ht[[SRC_LOC_COL, DST_LOC_COL,
                               "SOURCE_SS", "DESTINATION_SS" ,"FEEDERID_FULL", "DATECREATED" ,"COMMENTS"]]
                           .rename(columns={SRC_LOC_COL: "SOURCE_LOCATION",
                                            DST_LOC_COL: "DESTINATION_LOCATION"})
                           .drop_duplicates(),
                           how="left",
                           on=["SOURCE_LOCATION", "DESTINATION_LOCATION"]))

# 5️  EXPORT ------------------------------------------------------------------
out_cols = ["FEEDER_ID","FEEDERID_FULL",
            "FROM_SWITCH", "TO_SWITCH", 
            "SOURCE_SS", "DESTINATION_SS",
            "SOURCE_LOCATION", "DESTINATION_LOCATION",
            "RANK","DATECREATED", "COMMENTS"]

trace_df.to_csv(OUTPUT_PATH, index=False, columns=out_cols)
print(f"\nSaved {len(trace_df):,} rows → {OUTPUT_PATH}")

# Preview for interactive sessions
if __name__ == "__main__":
    try:
        from IPython.display import display
        display(trace_df.head())
    except Exception:
        pass



Saved 45,487 rows → /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/11_KV_FINAL_HEALTH/AFINAL.csv


,FEEDER_ID,FROM_SWITCH,TO_SWITCH,SOURCE_LOCATION,DESTINATION_LOCATION,RANK,SOURCE_SS,DESTINATION_SS,FEEDERID_FULL,DATECREATED,COMMENTS
0,15454,15454,38196,1S-MH-MU-ZST-RSTN-24TH,1S-MH-MU-ZST-CL02-1238,0,24TH ROAD REC-STN,GANGA JAMUNA SANGAM,24THRD_11KV_15454,2009-04-13 00:00:00+00:00,24TH ROAD REC-STN TO GANGA JAMUNA CHS (FROM SW...
1,15454,15454,38196,1S-MH-MU-ZST-RSTN-24TH,1S-MH-MU-ZST-CL02-1238,0,24TH ROAD REC-STN,GANGA JAMUNA SANGAM,24THRD_11KV_15454,2016-07-12 00:00:00+00:00,24TH ROAD REC-STN TO GANGA JAMUNA CHS (JT.NO.2...
2,15454,15454,38196,1S-MH-MU-ZST-RSTN-24TH,1S-MH-MU-ZST-CL02-1238,0,24TH ROAD REC-STN,GANGA JAMUNA SANGAM,24THRD_11KV_15454,2009-04-13 00:00:00+00:00,24TH ROAD REC-STN TO GANGA JAMUNA CHS (JT.NO.2...
3,15454,15454,38196,1S-MH-MU-ZST-RSTN-24TH,1S-MH-MU-ZST-CL02-1238,0,24TH ROAD REC-STN,GANGA JAMUNA SANGAM,24THRD_11KV_15454,2016-09-12 00:00:00+00:00,24TH ROAD REC-STN TO GANGA JAMUNA CHS (JT.NO.2...
4,15454,38195,34116,1S-MH-MU-ZST-CL02-1238,1S-MH-MU-ZST-CL02-0894,1,GANGA JAMUNA SANGAM,FORTUNE ENCLAVE,24THRD_11KV_15454,2016-09-12 00:00:00+00:00,GANGA JAMUNA CHS TO FORTUNE ENCLAVE (FROM SWNO...


next file generation

In [3]:
import pandas as pd
from pathlib import Path

# ── PATHS ────────────────────────────────────────────────────────────────────
AFINAL_PATH        = Path("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/11_KV_FINAL_HEALTH/AFINAL.csv")
ENERGY_AUDIT_PATH  = Path("/media/sagark24/New Volume/MERGE CDIS/2-Year-data/CLEANED_DATA/energyaudit_cleaned.csv")
FEEDERDETAIL_PATH  = Path("/media/sagark24/New Volume/MERGE CDIS/2-Year-data/CLEANED_DATA/feederdetails_cleaned.csv")
OUTPUT_CSV_PATH    = Path("/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/11_KV_FINAL_HEALTH/AFINAL_full.csv")

# ── 1) LOAD AFINAL -----------------------------------------------------------
df = pd.read_csv(AFINAL_PATH, low_memory=False)
df.columns = [c.upper() for c in df.columns]

# ── 2) SWITCH-LEVEL STATS (ENERGYAUDIT) -------------------------------------
audit = (pd.read_csv(
            ENERGY_AUDIT_PATH,
            usecols=["SWITCH_NO", "LOAD_FACTOR", "Y_INST_VOLTAGE", "CLUSTER_TYPE"],
            low_memory=False
         ).rename(str.upper, axis=1))

audit["SWITCH_NO"]      = audit["SWITCH_NO"].astype(str).str.strip()
audit["LOAD_FACTOR"]    = pd.to_numeric(audit["LOAD_FACTOR"],    errors="coerce")
audit["Y_INST_VOLTAGE"] = pd.to_numeric(audit["Y_INST_VOLTAGE"], errors="coerce")

def first_non_null(x):
    y = x.dropna()
    return y.iloc[0] if len(y) else None

switch_stats = (audit.dropna(subset=["SWITCH_NO"])
                     .groupby("SWITCH_NO")
                     .agg(
                         # LOAD_FACTOR
                         SWITCH_LOAD_FACTOR_MEAN   = ("LOAD_FACTOR", "mean"),
                         SWITCH_LOAD_FACTOR_MEDIAN = ("LOAD_FACTOR", "median"),
                         SWITCH_LOAD_FACTOR_STD    = ("LOAD_FACTOR", "std"),
                         SWITCH_LOAD_FACTOR_MIN    = ("LOAD_FACTOR", "min"),
                         SWITCH_LOAD_FACTOR_MAX    = ("LOAD_FACTOR", "max"),
                         # Y_INST_VOLTAGE
                         SWITCH_Y_INST_VOLTAGE_MEAN   = ("Y_INST_VOLTAGE", "mean"),
                         SWITCH_Y_INST_VOLTAGE_MEDIAN = ("Y_INST_VOLTAGE", "median"),
                         SWITCH_Y_INST_VOLTAGE_STD    = ("Y_INST_VOLTAGE", "std"),
                         SWITCH_Y_INST_VOLTAGE_MIN    = ("Y_INST_VOLTAGE", "min"),
                         SWITCH_Y_INST_VOLTAGE_MAX    = ("Y_INST_VOLTAGE", "max"),
                         # categorical
                         CLUSTER_TYPE = ("CLUSTER_TYPE", first_non_null)
                     )
                     .reset_index())

# switch_stats[["SWITCH_LOAD_FACTOR_STD","SWITCH_Y_INST_VOLTAGE_STD"]] = \
#     switch_stats[["SWITCH_LOAD_FACTOR_STD","SWITCH_Y_INST_VOLTAGE_STD"]].fillna(0)

df["FROM_SWITCH"] = df["FROM_SWITCH"].astype(str).str.strip()
df = (df.merge(switch_stats, how="left",
               left_on="FROM_SWITCH", right_on="SWITCH_NO")
        .drop(columns=["SWITCH_NO"]))

# ── 3) FEEDER-LEVEL STATS (FEEDERDETAILS) -----------------------------------
feeder = (pd.read_csv(
            FEEDERDETAIL_PATH,
            usecols=["SWITCHID", "FEEDERLOAD", "LOADFACTOR", "LOADLOSSFACTOR"],
            low_memory=False
         ).rename(str.upper, axis=1))

feeder["SWITCHID"]        = feeder["SWITCHID"].astype(str).str.strip()
for col in ["FEEDERLOAD", "LOADFACTOR", "LOADLOSSFACTOR"]:
    feeder[col] = pd.to_numeric(feeder[col], errors="coerce")

feeder_stats = (feeder.dropna(subset=["SWITCHID"])
                      .groupby("SWITCHID")
                      .agg(
                          # FEEDERLOAD
                          FEEDER_LOAD_MEAN   = ("FEEDERLOAD", "mean"),
                          FEEDER_LOAD_MEDIAN = ("FEEDERLOAD", "median"),
                          FEEDER_LOAD_STD    = ("FEEDERLOAD", "std"),
                          FEEDER_LOAD_MIN    = ("FEEDERLOAD", "min"),
                          FEEDER_LOAD_MAX    = ("FEEDERLOAD", "max"),
                          # LOADFACTOR
                          FEEDER_LOAD_FACTOR_MEAN   = ("LOADFACTOR", "mean"),
                          FEEDER_LOAD_FACTOR_MEDIAN = ("LOADFACTOR", "median"),
                          FEEDER_LOAD_FACTOR_STD    = ("LOADFACTOR", "std"),
                          FEEDER_LOAD_FACTOR_MIN    = ("LOADFACTOR", "min"),
                          FEEDER_LOAD_FACTOR_MAX    = ("LOADFACTOR", "max"),
                          # LOADLOSSFACTOR
                          FEEDER_LOSS_FACTOR_MEAN   = ("LOADLOSSFACTOR", "mean"),
                          FEEDER_LOSS_FACTOR_MEDIAN = ("LOADLOSSFACTOR", "median"),
                          FEEDER_LOSS_FACTOR_STD    = ("LOADLOSSFACTOR", "std"),
                          FEEDER_LOSS_FACTOR_MIN    = ("LOADLOSSFACTOR", "min"),
                          FEEDER_LOSS_FACTOR_MAX    = ("LOADLOSSFACTOR", "max")
                      )
                      .reset_index())

std_cols = [c for c in feeder_stats.columns if c.endswith("_STD")]
feeder_stats[std_cols] = feeder_stats[std_cols].fillna(0)

# ...  merge feeder_stats ---------------------------------------
df["FEEDER_ID"] = df["FEEDER_ID"].astype(str).str.strip()
df = (df.merge(feeder_stats, how="left",
               left_on="FEEDER_ID", right_on="SWITCHID")
        .drop(columns=["SWITCHID"]))

# ── KEEP FEEDER-STAT COLUMNS ONLY ON THE FIRST ROW OF EACH FEEDER ───────────
# (place this right after the merge with feeder_stats and BEFORE step 4)

# all columns that start with FEEDER_  *except* the ID itself
feeder_stat_cols = [
    c for c in df.columns
    if c.startswith("FEEDER_") and c != "FEEDER_ID"
]

# mask: True on very first row for each FEEDER_ID
first_row_mask = ~df.duplicated(subset="FEEDER_ID", keep="first")

# set stats to NA on later rows; FEEDER_ID column is left alone
df.loc[~first_row_mask, feeder_stat_cols] = pd.NA

# ── 4) APPEND GLOBAL MIN / MAX ROWS ----------------------------
num_cols = df.select_dtypes(include="number").columns
min_row  = df[num_cols].min().rename("GLOBAL_MIN")
max_row  = df[num_cols].max().rename("GLOBAL_MAX")


for col in df.columns:
    if col not in num_cols:
        min_row[col] = ""
        max_row[col] = ""

df_full = pd.concat([df, min_row.to_frame().T, max_row.to_frame().T],
                    ignore_index=True)

# ── 5) WRITE SINGLE CSV ------------------------------------------------------
df_full.to_csv(OUTPUT_CSV_PATH, index=False)
print(f"  All data   {OUTPUT_CSV_PATH}")


  All data   /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/11_KV_FINAL_HEALTH/AFINAL_full.csv


ADD NETWORKDETAILS 

In [4]:
import pandas as pd

csv_main    = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/11_KV_FINAL_HEALTH/AFINAL_full.csv"
csv_network = "/media/sagark24/New Volume/MERGE CDIS/2-Year-data/CLEANED_DATA/networkdetails_cleaned.csv"

df = pd.read_csv(csv_main, low_memory=False)
df.columns = [c.upper() for c in df.columns]

# ---------------------------------------------------------------------------
wanted = [
    "FROM_SWITCHID", "CABLESIZE",        # string
    "RESISTANCE", "LENGTH", "LENGTH_KM", # numeric → mean
    "NOOFJOINTS", "NOOFSUBSTATION",      # **counts → sum**
    "LTCURRENT", "SECTIONLOAD", "SECTIONLOSS_KW"
]

present  = pd.read_csv(csv_network, nrows=0).columns.str.upper()
use_cols = [c for c in wanted if c in present]

network = (pd.read_csv(csv_network, usecols=use_cols, low_memory=False)
             .rename(str.upper, axis=1))

network["FROM_SWITCHID"] = network["FROM_SWITCHID"].astype(str).str.strip()
df["FROM_SWITCH"]        = df["FROM_SWITCH"].astype(str).str.strip()

# force numerics where appropriate
num_try = ["RESISTANCE","LENGTH","LENGTH_KM",
           "LTCURRENT","SECTIONLOAD","SECTIONLOSS_KW"]
for col in num_try + ["NOOFJOINTS","NOOFSUBSTATION"]:
    if col in network.columns:
        network[col] = pd.to_numeric(network[col], errors="coerce")

# ----- aggregation rules ----------------------------------------------------
def first_non_null(s):
    x = s.dropna()
    return x.iloc[0] if len(x) else None

agg_dict = {
    # default rule for all numeric columns we coerced earlier
    **{c: "mean" for c in num_try if c in network.columns},
    # counts → SUM so they remain integers
    **{c: "sum"  for c in ["NOOFJOINTS","NOOFSUBSTATION"] if c in network.columns},
}

if "CABLESIZE" in network.columns:
    agg_dict["CABLESIZE"] = first_non_null

net_agg = (network.groupby("FROM_SWITCHID", dropna=False)
                  .agg(agg_dict)
                  .reset_index())

# join
df = (df.merge(net_agg, how="left",
               left_on="FROM_SWITCH", right_on="FROM_SWITCHID")
        .drop(columns=["FROM_SWITCHID"]))

# cast summed counts to nullable Int64 for cleanliness
for c in ["NOOFJOINTS","NOOFSUBSTATION"]:
    if c in df.columns:
        df[c] = df[c].round().astype("Int64")

# save
df.to_csv(csv_main, index=False)
print(f"  Updated file written  {csv_main}")

if "CABLESIZE" in df.columns:
    print("Rows with CABLESIZE filled:", df["CABLESIZE"].notna().sum())


  Updated file written  /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/11_KV_FINAL_HEALTH/AFINAL_full.csv
Rows with CABLESIZE filled: 35472


FAULT DATA 

In [5]:
import pandas as pd
from collections import Counter

# ── FILE LOCATIONS ───────────────────────────────────────────────────────────
CABLE_PATH  = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/11_KV_FINAL_HEALTH/AFINAL_full.csv"
FAULT_PATH  = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/DATA_GENERATION/HT_fault_cable_info_processed2.csv"

OUT_PATH    = CABLE_PATH   # overwrite; change if you want a new file
# FEEDER_SUMMARY_PATH = "/media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/11_KV_FINAL_HEALTH/FEEDER_FAULT_SUMMARY.csv"

# ── 1) LOAD ------------------------------------------------------------------
cables = pd.read_csv(CABLE_PATH, low_memory=False)
faults = pd.read_csv(FAULT_PATH, low_memory=False)

cables.columns = [c.upper() for c in cables.columns]
faults.columns = [c.upper() for c in faults.columns]

# ── 2) NORMALISE ID STRINGS --------------------------------------------------
def norm_id(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.strip()
         .str.upper()
         .str.replace(r"\.0+$", "", regex=True)        # 15454.0 → 15454
         .str.replace(r"[^A-Z0-9]", "", regex=True)    # keep A-Z,0-9
         .str.lstrip("0")                              # drop leading zeros
    )

cables["FROM_SWITCH_N"] = norm_id(cables["FROM_SWITCH"])
cables["FEEDER_ID_N"]   = norm_id(cables["FEEDER_ID"])
faults["FROM_SWITCH_N"] = norm_id(faults["FROM_SWITCH"])
faults["SWITCH_NO_N"]   = norm_id(faults["SWITCH_NO"])

# ── 3) PREP FAULT NUMERICS ---------------------------------------------------
faults["TIME_OUTAGE"]           = pd.to_datetime(faults["TIME_OUTAGE"], errors="coerce")
faults["TIME_DIFFERENCE_HOURS"] = pd.to_numeric(faults["TIME_DIFFERENCE_HOURS"], errors="coerce")

def mode(series):
    nn = series.dropna()
    return Counter(nn).most_common(1)[0][0] if len(nn) else None

# ── 4-A) CABLE-LEVEL STATS  (prefix CBL_) -----------------------------------
cbl_stats = (
    faults.groupby("FROM_SWITCH_N")
          .agg(
              CBL_FAULT_COUNT          = ("FROM_SWITCH_N", "size"),
              CBL_AVG_REPAIR_HRS       = ("TIME_DIFFERENCE_HOURS", "mean"),
              CBL_MAX_REPAIR_HRS       = ("TIME_DIFFERENCE_HOURS", "max"),
              CBL_LATEST_OUTAGE        = ("TIME_OUTAGE", "max"),
              CBL_COMMON_REASON_CAT    = ("REASON_CATEGORY", mode),
              CBL_COMMON_REASON_TEXT   = ("REASON_TEXT",    mode),
              CBL_COMMON_RELAY_FUSE    = ("RELAY_FUSE",     mode),
          )
          .reset_index()
)
cbl_stats[["CBL_AVG_REPAIR_HRS","CBL_MAX_REPAIR_HRS"]] = (
    cbl_stats[["CBL_AVG_REPAIR_HRS","CBL_MAX_REPAIR_HRS"]].round(2)
)

# ── 4-B) FEEDER-LEVEL STATS  (prefix FDR_) ----------------------------------
fdr_stats = (
    faults.groupby("SWITCH_NO_N")
          .agg(
              FDR_FAULT_COUNT          = ("SWITCH_NO_N", "size"),
              FDR_AVG_REPAIR_HRS       = ("TIME_DIFFERENCE_HOURS", "mean"),
              FDR_MAX_REPAIR_HRS       = ("TIME_DIFFERENCE_HOURS", "max"),
              FDR_FIRST_OUTAGE         = ("TIME_OUTAGE", "min"),
              FDR_LAST_OUTAGE          = ("TIME_OUTAGE", "max"),
              FDR_COMMON_REASON_CAT    = ("REASON_CATEGORY", mode),
              FDR_COMMON_REASON_TEXT   = ("REASON_TEXT",    mode),
              FDR_COMMON_RELAY_FUSE    = ("RELAY_FUSE",     mode),
          )
          .reset_index()
)
fdr_stats[["FDR_AVG_REPAIR_HRS","FDR_MAX_REPAIR_HRS"]] = (
    fdr_stats[["FDR_AVG_REPAIR_HRS","FDR_MAX_REPAIR_HRS"]].round(2)
)

# ── 5) MERGE NEW COLUMNS -----------------------------------------------------
# (we never drop existing columns; we just add new ones)
cables = cables.merge(cbl_stats, how="left", on="FROM_SWITCH_N")
cables = cables.merge(
            fdr_stats.rename(columns={"SWITCH_NO_N": "FEEDER_ID_N"}),
            how="left", on="FEEDER_ID_N"
)

# ── 6) KEEP FEEDER_… COLUMNS ONLY ON FIRST CABLE OF EACH FEEDER -------------
feeder_cols = [c for c in cables.columns if c.startswith("FDR_")]
first_row_mask = ~cables.duplicated(subset="FEEDER_ID_N", keep="first")
cables.loc[~first_row_mask, feeder_cols] = pd.NA   # blanks after first row

# ── 7) CLEAN-UP & SAVE -------------------------------------------------------
cables = cables.drop(columns=["FROM_SWITCH_N","FEEDER_ID_N"])
cables.to_csv(OUT_PATH, index=False)
# fdr_stats.rename(columns={"SWITCH_NO_N":"FEEDER_ID"}).to_csv(FEEDER_SUMMARY_PATH, index=False)

print(" cable rows with CBL_FAULT_COUNT :", cables['CBL_FAULT_COUNT'].notna().sum())
print(" feeders with FDR_FAULT_COUNT    :", fdr_stats.shape[0])
print(" updated file                    :", OUT_PATH)
# print(" feeder summary                  :", FEEDER_SUMMARY_PATH)


 cable rows with CBL_FAULT_COUNT : 16032
 feeders with FDR_FAULT_COUNT    : 1543
 updated file                    : /media/sagark24/New Volume/MERGE CDIS/IPYNB_FILE/11_KV_FINAL_HEALTH/AFINAL_full.csv


In [7]:
import streamlit as st
import streamlit_cytoscapejs as cyto

elements = [
    {"data": {"id": "one", "label": "Node 1"}},
    {"data": {"id": "two", "label": "Node 2"}},
    {"data": {"source": "one", "target": "two", "label": "Edge from 1 to 2"}}
]

selected = cyto.st_cytoscapejs(
    elements=elements,
    layout={'name': 'cose'},
    width='100%',
    height='500px',
    key="cytoscape"
)

if selected:
    st.write("Selected:", selected)


TypeError: st_cytoscapejs() got an unexpected keyword argument 'layout'